## XG-Boost

---
### parameter tuning
* 일반 파라미터
부스팅을 수행할 때 트리를 사용할지, 선형 모델을 사용할지 등을 고른다.
* 부스터 파라미터
선택한 부스터에 따라서 적용할 수 있는 파라미터 종류가 다르다.
* 학습 과정 파라미터
학습 시나리오를 결정한다.

* 일반 파라미터
    - booster [기본값 = gbtree]
        어떤 부스터 구조를 쓸지 결정한다.
        의사결정기반모형(gbtree), 선형모형(gblinear), dart가 있다.
    - n_jobs
        XGBoost를 실행하는 데 사용되는 병렬 스레드 수
    - verbosity [기본값 = 1]
        유효한 값은 0 (무음), 1 (경고), 2 (정보), 3 (디버그)
        
        
* 부스터 파라미터
    - gbtree Booster의 파라미터
        - learning_rate [ 기본값 : 0.3 ]
            learning rate가 높을수록 과적합 하기 쉽다.
        - n_estimators [ 기본값 : 100 ]
            생성할 weak learner의 수
            learning_rate가 낮을 땐, n_estimators를 높여야 과적합이 방지된다.
        - max_depth [ 기본값 : 6 ]
            트리의 maximum depth이다.
            적절한 값이 제시되어야 하고 보통 3-10 사이 값이 적용된다.
            max_depth가 높을수록 모델의 복잡도가 커져 과적합 하기 쉽다.
        - min_child_weight [ 기본값 : 1 ]
            관측치에 대한 가중치 합의 최소를 말한다.
            값이 높을수록 과적합이 방지된다.
        - gamma [ 기본값 : 0 ]
            리프노드의 추가분할을 결정할 최소손실 감소값이다.
            해당값보다 손실이 크게 감소할 때 분리한다.
            값이 높을수록 과적합이 방지된다.
        - subsample [ 기본값 : 1 ]
            weak learner가 학습에 사용하는 데이터 샘플링 비율이다.
            보통 0.5 ~ 1 사용된다.
            값이 낮을수록 과적합이 방지된다.
        - colsample_bytree [ 기본값 : 1 ]
            각 tree 별 사용된 feature의 퍼센테이지이다.
            보통 0.5 ~ 1 사용된다.
            값이 낮을수록 과적합이 방지된다.
        - lambda [기본값 = 1, 별칭 : reg_lambda]
            가중치에 대한 L2 Regularization 적용 값
            피처 개수가 많을 때 적용을 검토
            이 값이 클수록 과적합 감소 효과
        - alpha [기본값 = 0, 별칭 : reg_alpha]
            가중치에 대한 L1 Regularization 적용 값
            피처 개수가 많을 때 적용을 검토
            이 값이 클수록 과적합 감소 효과

* 학습 과정 파라미터
    - objective [ 기본값 : reg = squarederror ]
        - reg : squarederror [제곱 손실이 있는 회귀 ]
        - binary : logistic (binary-logistic classification)
        [이항 분류 문제 로지스틱 회귀 모형으로 반환값이 클래스가 아니라 예측 확률]
        - multi : softmax 
        다항 분류 문제의 경우 소프트맥스(Softmax)를 사용해서 분류하는데 반횐되는 값이 예측확률이 아니라 클래스임. 또한 num_class도 지정해야함.
        - multi : softprob
        클래스 범주에 속하는 예측확률을 반환함.
        - count : poisson (count data poison regression) 등 다양하다.
    - eval_metric : 모델의 평가 함수를 조정하는 함수다.
    설정한 objective 별로 기본설정값이 지정되어 있다.
        - rmse: root mean square error
        - mae: mean absolute error
        - logloss: negative log-likelihood
        - error: Binary classification error rate (0.5 threshold)
        - merror: Multiclass classification error rate
        - mlogloss: Multiclass logloss
        - auc: Area under the curve
        map (mean average precision)등, 해당 데이터의 특성에 맞게 평가 함수를 조정한다.            
    - seed [ 기본값 : 0 ]
        재현가능하도록 난수를 고정시킴.

* 민감하게 조정해야하는 것

    - booster 모양
    - eval_metric(평가함수) / objective(목적함수)
    - eta 
    - L1 form (L1 레귤러라이제이션 폼이 L2보다 아웃라이어에 민감하다.)
    - L2 form

* 과적합 방지를 위해 조정해야하는 것

    - learning rate 낮추기 → n_estimators은 높여야함
    - max_depth 낮추기
    - min_child_weight 높이기
    - gamma 높이기
    - subsample, colsample_bytree 낮추기


---
### 참고
* https://www.kaggle.com/lifesailor/xgboost
* https://wooono.tistory.com/97


In [10]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import plot_importance

import optuna 
from optuna import Trial, visualization
from optuna.samplers import TPESampler

#autogloun

%matplotlib inline

In [11]:
import warnings

warnings.filterwarnings( 'ignore' )

In [12]:
train_data = pd.read_csv("train.csv") 
test_data = pd.read_csv("test.csv")

In [13]:
x_data = train_data.loc[:, 'f0':'f99']
y_data = train_data.loc[:, 'loss']

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
x_train, x_test, y_train, y_test=train_test_split(x_data,
                                                  y_data,
                                                  test_size=0.2,   #전체 중 20%를 테스트용으로 분할
                                                                   #나머지 80%는 훈련용
                                                  shuffle=True,    #무작위로 섞어서 추출
                                                  random_state=20) #무작위 추출 시 일정한 기준으로

In [ ]:
import xgboost as xgb

# 모델 선언
my_model = xgb.XGBRegressor(
    verbosity = 2
    learning_rate=0.1,
    max_depth=5,
    n_estimators=100)

# 모델 훈련
my_model.fit(x_train, y_train, verbose=False)

# 모델 예측
y_test = my_model.predict(x_test) 

In [24]:
import xgboost as xgb

# 모델 선언
my_model = xgb.XGBRegressor(
    n_estimators = 3520,
    max_depth = 11,
    min_child_weight = 231,
    gamma = 2,
    colsample_bytree = 0.7,
    reg_lambda = 0.014950936465569798,
    alpha = 0.28520156840812494,
    subsample = 0.6,
    learning_rate=0.01)

# 모델 훈련
my_model.fit(x_train, y_train, verbose=False)

# 모델 예측
y_pred = my_model.predict(x_test) 

In [25]:
rms = np.sqrt(mean_squared_error(y_pred, y_test))
print(rms)

7.797709619810067


In [ ]:
def objectiveXGB(trial: Trial, X, y, test):
    param = {
        "n_estimators" : trial.suggest_int('n_estimators', 100, 10000),
        'max_depth':trial.suggest_int('max_depth', 5, 10),
        'min_child_weight':trial.suggest_int('min_child_weight', 1, 300),
        'gamma':trial.suggest_int('gamma', 1, 3),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.3, 0.1, 0.05, 0.01]),
        'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree',0.5, 1, 0.1),
        'tree_method': 'hist',
        'predictor': 'gpu_predictor',
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'subsample': trial.suggest_categorical('subsample', [0.6,0.7,0.8,1.0] ),
        'random_state': 42
    }
    X_train, X_test, y_train, y_test = train_test_split(X, y.flatten(), test_size=0.1)
    
    y_train = y_train.reshape(-1, 1)
    y_test  = y_test.reshape(-1, 1)

    model = xgb.XGBRegressor(**param)
    xgb_model = model.fit(X_train, y_train, verbose=False, eval_set=[(X_test, y_test)])
    score = mean_squared_error(xgb_model.predict(X_test), y_test, squared=False)

    return score

In [ ]:
study = optuna.create_study(direction='minimize',
                            sampler=TPESampler())
study.optimize(lambda trial : objectiveXGB(trial, X,  y, X_test), n_trials=50)
print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

best_param = study.best_trial.params
xgbReg = train(xgb.XGBRegressor(**best_param, tree_method='gpu_hist', random_state=42, 
                                predictor='gpu_predictor', learning_rate=0.01, nthread=-1))

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_param_importances(study)